Discrete POMDP implemented using `github.com/JuliaPOMDP/QuickPOMDPs.jl` and tutorial [here](https://htmlview.glitch.me/?https://github.com/JuliaAcademy/Decision-Making-Under-Uncertainty/blob/master/html/2-POMDPs.jl.html).

In [1]:
using POMDPs, QuickPOMDPs, POMDPModelTools, POMDPPolicies, Parameters, Random, Plots, LinearAlgebra
using DiscreteValueIteration, TabularTDLearning, POMDPTools, QMDP, POMCPOW

In [2]:
# check Julia exists
print("Hello World")

Hello World

In [3]:
@with_kw struct MyParameters
    N::Int = 4   # size of item set
    K::Int = 3   # size of arm set
    M::Int = 2   # size of beta set
    y::Real = 1.0 # discount factor
    umax::Real = 10  # max utility
end

params = MyParameters()

MyParameters
  N: Int64 4
  K: Int64 3
  M: Int64 2
  y: Float64 1.0
  umax: Int64 10


In [4]:
struct State
    u::Array{Int}          # list of N utility values for N items
    d::Array{Array{Real}}  # list of K arm distributions, each assigning probabilities to N items
    b::Array{Float64}      # list of M beta values
end

In [5]:
# space of possible utility functions (each must be length N)
umax = 10
U = [[params.umax, 0, 0, 0],
    [0, params.umax, 0, 0],
    [0, 0, params.umax, 0],
    [0, 0, 0, params.umax]]

# space of possible arm distributions (each must be size KxN)
D = [[[1/params.N, 1/params.N, 1/params.N, 1/params.N], [1, 0, 0, 0], [0.5, 0.5, 0, 0]],
    [[0, 0, 0, 1], [1/params.N, 1/params.N, 1/params.N, 1/params.N], [0.5, 0.5, 0, 0]]]

# beta values (must be length M)
B = [0.1,10]

# State space
S = [[State(u,d,B) for u in U, d in D]...,]

# state comparator
Base.:(==)(s1::State, s2::State) = (s1.u == s2.u) && (s1.d == s2.d)

In [6]:
# Action space - actions are arm choices (K) or beta selections (M)
@enum Action C1 C2 C3 B1 B2
A = [C1, C2, C3, B1, B2]

5-element Vector{Action}:
 C1::Action = 0
 C2::Action = 1
 C3::Action = 2
 B1::Action = 3
 B2::Action = 4

In [7]:
# Transition function
function T(s::State, a::Action)
    return SparseCat([s], [1.0])    # categorical distribution
end

T (generic function with 1 method)

In [8]:
# Reward function
function R(s::State, a::Action)
    # if arm pulled, return that arm's avg utility
    if a == C1
        utilities = s.u
        arm_dist = s.d[1]
        return dot(utilities, arm_dist)
    elseif a == C2
        utilities = s.u
        arm_dist = s.d[2]
        return dot(utilities, arm_dist)
    elseif a == C3
        utilities = s.u
        arm_dist = s.d[3]
        return dot(utilities, arm_dist)
    # if beta selected, return 0
    else
        return 0
    end
end

R (generic function with 1 method)

In [9]:
# item space
I = 1:params.N

# preference space
struct Preference
    i0::Int    # first item to compare, in {1,2,...,N}
    i1::Int    # second item to compare, in {1,2,...,N}
    label::Int # feedback label, in {0,1}
end

P = [[Preference(i0,i1,label) for i0 in I, i1 in I, label in [0,1]]...,]

# observation space
struct Observation
    isItem::Bool    # true if item returned, false otherwise
    i::Int          # item, if item returned
    p::Preference   # preference, if preference returned
end

invalid_i = -1
invalid_p = Preference(-1,-1,-1)
I_obs = [Observation(true, i, invalid_p) for i in I]
P_obs = [Observation(false, invalid_i, p) for p in P]
omega = union(I_obs, P_obs)

36-element Vector{Observation}:
 Observation(true, 1, Preference(-1, -1, -1))
 Observation(true, 2, Preference(-1, -1, -1))
 Observation(true, 3, Preference(-1, -1, -1))
 Observation(true, 4, Preference(-1, -1, -1))
 Observation(false, -1, Preference(1, 1, 0))
 Observation(false, -1, Preference(2, 1, 0))
 Observation(false, -1, Preference(3, 1, 0))
 Observation(false, -1, Preference(4, 1, 0))
 Observation(false, -1, Preference(1, 2, 0))
 Observation(false, -1, Preference(2, 2, 0))
 Observation(false, -1, Preference(3, 2, 0))
 Observation(false, -1, Preference(4, 2, 0))
 Observation(false, -1, Preference(1, 3, 0))
 ⋮
 Observation(false, -1, Preference(1, 2, 1))
 Observation(false, -1, Preference(2, 2, 1))
 Observation(false, -1, Preference(3, 2, 1))
 Observation(false, -1, Preference(4, 2, 1))
 Observation(false, -1, Preference(1, 3, 1))
 Observation(false, -1, Preference(2, 3, 1))
 Observation(false, -1, Preference(3, 3, 1))
 Observation(false, -1, Preference(4, 3, 1))
 Observation(fal

In [10]:
# query profile (likelihood of querying 1,1; 2,1; 3,1; ... ; N,1; 1,2; 2,2; ... ; N,N)
# not neccessarily normalized
Q = ones(params.N*params.N)

# preference probability (expected preference, or probability that preference=1)
function Pr(p::Preference, s::State, b::Float64)
    prob_pref_1 = exp(Float64(b)*s.u[p.i1])/(exp(Float64(b)*s.u[p.i1])+exp(Float64(b)*s.u[p.i0]))
    if p.label == 1
        return prob_pref_1
    else
        return 1.0-prob_pref_1
    end
end

function O(s::State, a::Action, sp::State)
    # if C action, obs in I_obs
    A_c = [C1, C2, C3]
    if a in A_c
        arm_choice = Int(a)+1   # adding 1 b/c enums index from 0
        return SparseCat(I_obs, s.d[arm_choice])
    end
    
    # if B action, obs in P_obs
    A_b = [B1, B2]
    if a in A_b
        b = Int(a)+1-length(A_c)
        prob_of_pref = [Pr(o.p, s, s.b[b]) for o in P_obs]
        prob_of_query = vcat(Q,Q)   # doubled because each query appears once for each label
        
        # weight by querying profile to get dist
        dist = [prob_of_pref[i]*prob_of_query[i] for i in 1:length(prob_of_pref)]
        normalized_dist = dist/sum(dist)        
        return SparseCat(P_obs, normalized_dist)
    end
end

O (generic function with 1 method)

In [11]:
# define POMDP
abstract type MyPOMDP <: POMDP{State, Action, Observation} end
pomdp = QuickPOMDP(MyPOMDP,
    states       = S,
    actions      = A,
    observations = omega,
    transition   = T,
    observation  = O,
    reward       = R,
    discount     = params.y,
    initialstate = S);

In [12]:
policy = RandomPolicy(pomdp)

for (s,a,r,o) in stepthrough(pomdp, policy, "s,a,r,o", max_steps=3)
    @show s
    @show a
    @show r
    @show o
    println()
end

s = State([0, 0, 10, 0], Array{Real}[[0.0, 0.0, 0.0, 1.0], [0.25, 0.25, 0.25, 0.25], [0.5, 0.5, 0.0, 0.0]], [0.1, 10.0])
a = C2
r = 2.5
o = Observation(true, 4, Preference(-1, -1, -1))

s = State([0, 0, 10, 0], Array{Real}[[0.0, 0.0, 0.0, 1.0], [0.25, 0.25, 0.25, 0.25], [0.5, 0.5, 0.0, 0.0]], [0.1, 10.0])
a = B2
r = 0
o = Observation(false, -1, Preference(4, 3, 1))

s = State([0, 0, 10, 0], Array{Real}[[0.0, 0.0, 0.0, 1.0], [0.25, 0.25, 0.25, 0.25], [0.5, 0.5, 0.0, 0.0]], [0.1, 10.0])
a = C3
r = 0.0
o = Observation(true, 2, Preference(-1, -1, -1))



In [15]:
@time begin
    solver = POMCPOWSolver()
    planner = solve(solver, pomdp);
end
println("Solved")

rollout = true

# if rollout
#     show_state = true
#     for (s,a,r,o) in stepthrough(pomdp, planner, "s,a,r,o", max_steps=3)
#         if show_state
#             @show s
#             println("")
#             show_state = false
#         end
#         @show a
#         @show r
#         @show o
#         println()
#     end
# end

  0.000024 seconds (12 allocations: 784 bytes)
Solved


true

In [14]:
for iter in [1, 10, 100, 1000, 10000, 100000, 1000000, 10000000]
    
    @time begin
        solver = Solver(max_iterations=iter);
        policy = solve(solver, pomdp)
        val = value(policy, b)
    end
    
    println("iter: ", iter)
    println("val: ", val,"\n")
end

println("done")

LoadError: MethodError: no constructors have been defined for Solver